# Satellite Imagery Composition

This notebook is dedicated to building a multi-layer composite image for the Alaska Pollock fishery study.  
It includes the retrieval, processing, and combination of environmental and satellite-derived variables using Google Earth Engine.

The resulting composite will be used as the foundation for feature extraction, training, and spatial prediction in a separate modeling notebook.


## Step 0: Install Required Packages

This notebook is dedicated to **satellite image preparation and environmental variable composition** using Google Earth Engine.

To enable this, we install a few geospatial and Earth Engine-related Python packages:

- `earthengine-api` and `geemap`: for interacting with Earth Engine and visualizing results
- `geopandas` and `matplotlib`: for handling geospatial vector data and creating plots (used occasionally for diagnostics)
- `cartopy`: for cartographic projections (when needed)
- `openpyxl`: for Excel export compatibility (if results need to be saved locally)

> These installations are only required once per Colab session. If you're running this locally, ensure these packages are installed in your environment.



In [1]:
# Install Earth Engine API and visualization tools
!pip install earthengine-api
!pip install geemap

# Install system dependencies for geospatial projections
!apt-get install -y python3-cartopy python3-pip

# Install core geospatial and plotting libraries
!pip install geopandas matplotlib

# Install Excel I/O support
!pip install openpyxl



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-lyx libimagequant0 liblbfgsb0 libraqm0 libxsimd-dev
  python-cartopy-data python-matplotlib-data python3-appdirs python3-attr
  python3-beniget python3-brotli python3-bs4 python3-certifi python3-chardet
  python3-cycler python3-dateutil python3-decorator python3-fonttools
  python3-fs python3-gast python3-html5lib python3-kiwisolver python3-lxml
  python3-lz4 python3-matplotlib python3-mpmath python3-numpy python3-olefile
  python3-packaging python3-pil python3-pil.imagetk python3-ply python3-pyproj
  python3-pyshp python3-pythran python3-scipy python3-setuptools
  python3-shapely python3-soupsieve python3-sympy python3-tz python3-ufolib2
  python3-unicodedata2 python3-webencodings python3-wheel unicode-data
Suggested packages:
  libxsim

## ⚙️ Step 1: Authenticate Earth Engine

This step ensures access to **Google Earth Engine (GEE)** for retrieving and processing satellite data.

To use Earth Engine in this notebook:

1. You must have an account at [signup.earthengine.google.com](https://signup.earthengine.google.com).
2. Visit [code.earthengine.google.com](https://code.earthengine.google.com), and in the left panel:
   - Click your username
   - Select **"Add a new Cloud Project"**
   - Create or select a Google Cloud project
   - Copy the **Project ID** (e.g. `my-gee-project-123456`)
   - Paste it into the code block below

Once authenticated and initialized, you will be able to run all satellite imagery operations directly through Earth Engine in this notebook.


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-batistahpedro')


## 🛰️ Step 2: Create Satellite Image Composite with Google Earth Engine

In this step, we generate satellite image composites using Google Earth Engine (GEE).  
These composites will serve as sources of explanatory environmental variables to support the modeling of fishing effort and catch for the Alaska Pollock fishery.

The general workflow includes:

1. Selecting a satellite image collection (e.g., MODIS, VIIRS)  
2. Filtering by spatial extent (AOI), date range, and optionally by cloud cover  
3. Preprocessing the imagery (e.g., band selection, cloud masking, scaling)  
4. Aggregating images into a composite using statistical reducers (e.g., median, maximum, mosaic)

> ⚠️ This step assumes GEE authentication and Drive mounting are already completed (see Step 1).



In [3]:
# ===============================================
# ⚙️ Global Parameters for Satellite Image Composites
# ===============================================
# These parameters are reused across all satellite products.
# They are set to match the spatial and temporal resolution of your analysis.

# Area of Interest (AOI): Expanded polygon covering the Bering Sea and Aleutian region
# Note: defined directly in EPSG:4326 (lat/lon) for full compatibility with Earth Engine
aoi = ee.Geometry.Polygon([
    [[150, 45], [150, 70], [-150, 70], [-150, 45], [150, 45]]
])

# Projection used across the analysis: EPSG:4326 (lat/lon)
crs_code = 'EPSG:4326'

# Temporal range used in all image composites
start_date = '2012-01-01'
end_date = '2012-12-31'

# Spatial resolution in meters (e.g., MODIS: 1000m, Sentinel-2: 30m, Landsat: 30m)
scale_value = 1000


### 🌐 2.1 Generate MODIS Surface Reflectance Composite (2012)

In this sub-step, we create a composite from the MODIS Surface Reflectance product (`MODIS/006/MODOCGA`) for the year 2012.  
This composite provides important surface reflectance features (e.g., visible and infrared bands) for spatial modeling.

The process includes:

1. Defining the area of interest (AOI) and reprojecting it to EPSG:3832  
2. Filtering MODIS imagery by time range and spatial bounds  
3. Selecting reflectance bands of interest  
4. Computing pixel-level statistics: **median**, **maximum**, and **minimum**  
5. Scaling reflectance values using the MODIS scale factor (0.0001)  
6. Reprojecting each band to a consistent CRS and resolution (EPSG:3832, 1 km)  
7. Merging results into a single composite  
8. Visualizing the outputs using `geemap`


In [4]:
import ee
import geemap

# 1. Load MODIS Surface Reflectance image collection
modis_sr = ee.ImageCollection("MODIS/006/MODOCGA")

# 2. Define reflectance bands of interest
bands = [
    'sur_refl_b08', 'sur_refl_b09', 'sur_refl_b10', 'sur_refl_b11',
    'sur_refl_b12', 'sur_refl_b13', 'sur_refl_b14', 'sur_refl_b15', 'sur_refl_b16'
]

# 3. Filter collection by time and select bands
sr_2012 = modis_sr.filterDate(start_date, end_date).select(bands)

# 4. Compute pixel-wise statistics and clip to AOI (no reprojection)
sr_median = sr_2012.median().clip(aoi).rename([f"{b}_median" for b in bands])
sr_max = sr_2012.max().clip(aoi).rename([f"{b}_max" for b in bands])
sr_min = sr_2012.min().clip(aoi).rename([f"{b}_min" for b in bands])

# 5. Apply MODIS scale factor (scaled integers → reflectance in [0–1])
scale_factor = 0.0001
sr_median_scaled = sr_median.multiply(scale_factor)
sr_max_scaled = sr_max.multiply(scale_factor)
sr_min_scaled = sr_min.multiply(scale_factor)

# 6. (Optional) Resample using bicubic interpolation
sr_median_scaled = sr_median_scaled.resample('bicubic')
sr_max_scaled = sr_max_scaled.resample('bicubic')
sr_min_scaled = sr_min_scaled.resample('bicubic')

# 7. Combine all scaled layers into a single image
sr_composite = ee.Image.cat([sr_median_scaled, sr_max_scaled, sr_min_scaled])

# 8. Define visualization parameters for reflectance
vis_params = {
    'min': 0.0,
    'max': 0.3,
    'palette': ['black', 'blue', 'green', 'yellow', 'red', 'white']
}

# 9. Initialize and display interactive map
Map_Final = geemap.Map()
Map_Final.centerObject(aoi, zoom=4)

# 10. Add each band’s stats to the map
for band in bands:
    Map_Final.addLayer(sr_median_scaled.select(f"{band}_median"), vis_params, f"MODIS {band} Median (2012)")
    Map_Final.addLayer(sr_max_scaled.select(f"{band}_max"), vis_params, f"MODIS {band} Max (2012)")
    Map_Final.addLayer(sr_min_scaled.select(f"{band}_min"), vis_params, f"MODIS {band} Min (2012)")

Map_Final.addLayerControl()
Map_Final


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🌊 2.2 Generate MODIS-Aqua Ocean Color Composite (2012)

In this sub-step, we create a composite from the MODIS-Aqua Ocean Color SMI (Standard Mapped Image) product (`NASA/OCEANDATA/MODIS-Aqua/L3SMI`) for the year 2012.  
This composite provides oceanographic features relevant to fisheries, such as chlorophyll-a concentration, light attenuation, particulate organic carbon (POC), and sea surface temperature (SST).

The process includes:

1. Filtering the MODIS-Aqua dataset by date and AOI  
2. Selecting bands related to ocean color and biological productivity  
3. Computing **median**, **maximum**, and **minimum** composites  
4. Reprojecting all bands to a common CRS and resolution  
5. Merging the statistics into a single image  
6. Visualizing selected variables interactively with appropriate color palettes


In [5]:
import ee
import geemap

# 1. Load MODIS-Aqua Ocean Color dataset
modis_aqua = ee.ImageCollection("NASA/OCEANDATA/MODIS-Aqua/L3SMI")

# 2. Select relevant bands
bands = [
    'chlor_a', 'nflh', 'poc', 'sst',
    'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488',
    'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645',
    'Rrs_667', 'Rrs_678'
]

# 3. Filter and select for 2012
aqua_2012 = modis_aqua.filterDate(start_date, end_date).select(bands)

# 4. Compute per-pixel statistics and clip to AOI
aqua_median = aqua_2012.median().clip(aoi).rename([f"MODIS_Aqua_{b}_median" for b in bands])
aqua_max = aqua_2012.max().clip(aoi).rename([f"MODIS_Aqua_{b}_max" for b in bands])
aqua_min = aqua_2012.min().clip(aoi).rename([f"MODIS_Aqua_{b}_min" for b in bands])

# 5. Optional: smooth values
aqua_median = aqua_median.resample('bicubic')
aqua_max = aqua_max.resample('bicubic')
aqua_min = aqua_min.resample('bicubic')

# 6. Combine into composite
aqua_composite = ee.Image.cat([aqua_median, aqua_max, aqua_min])

# 7. Visualization parameters
vis_params = {
    'chlor_a': {'min': 0, 'max': 99.99, 'palette': ['blue', 'green', 'yellow', 'orange', 'red']},
    'nflh': {'min': 0, 'max': 5, 'palette': ['black', 'blue', 'green', 'yellow', 'red', 'white']},
    'poc': {'min': 0, 'max': 13000, 'palette': ['black', 'purple', 'blue', 'green', 'yellow', 'red']},
    'sst': {'min': -2, 'max': 35, 'palette': ['darkblue', 'blue', 'cyan', 'green', 'yellow', 'orange', 'red']}
}

# 8. Initialize map
Map_Final = geemap.Map()
Map_Final.centerObject(aoi, 4)

# 9. Add bands to map
for band in bands:
    for stat in ['median', 'max', 'min']:
        layer_name = f"MODIS-Aqua {band} {stat.capitalize()} (2012)"
        band_name = f"MODIS_Aqua_{band}_{stat}"
        palette = vis_params.get(band, {'min': 0, 'max': 0.08, 'palette': ['black', 'blue', 'cyan', 'green', 'yellow', 'red']})
        Map_Final.addLayer(eval(f"aqua_{stat}").select(band_name), palette, layer_name)

Map_Final.addLayerControl()
Map_Final


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🌬️ 2.3 Generate NOAA CDR Atmospheric Composite (2012)

In this sub-step, we use the **NOAA Climate Data Record (CDR)** of Near-Surface Atmospheric Properties  
(`NOAA/CDR/ATMOS_NEAR_SURFACE/V2`) to generate a composite for 2012.  

This dataset includes large-scale atmospheric variables that influence ocean conditions and biological productivity, such as:

- Air temperature  
- Specific humidity  
- Wind speed

The process includes:

1. Filtering the NOAA CDR dataset by date and region  
2. Selecting atmospheric bands of interest  
3. Computing **median**, **maximum**, and **minimum** composites  
4. Reprojecting bands to a common CRS and resolution  
5. Merging into a single composite  
6. Dynamically adjusting visualization based on data ranges  
7. Displaying results using `geemap`


In [6]:
import ee
import geemap

# 1. Load NOAA CDR Near-Surface Atmospheric dataset
atmos_data = ee.ImageCollection("NOAA/CDR/ATMOS_NEAR_SURFACE/V2")

# 2. Select relevant bands
bands = ['air_temperature', 'specific_humidity', 'wind_speed']

# 3. Filter by date and select bands
atmos_2012 = atmos_data.filterDate(start_date, end_date).select(bands)

# 4. Compute per-pixel statistics (median, max, min) and clip to AOI
atmos_median = atmos_2012.median().clip(aoi).rename([f"NOAA_{b}_median" for b in bands])
atmos_max = atmos_2012.max().clip(aoi).rename([f"NOAA_{b}_max" for b in bands])
atmos_min = atmos_2012.min().clip(aoi).rename([f"NOAA_{b}_min" for b in bands])

# 5. Optional: Resample for smooth visualization
atmos_median = atmos_median.resample('bicubic')
atmos_max = atmos_max.resample('bicubic')
atmos_min = atmos_min.resample('bicubic')

# 6. Combine into composite
atmos_composite = ee.Image.cat([atmos_median, atmos_max, atmos_min])

# 7. Dynamic stats for visualization
stats = atmos_median.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=scale_value,
    bestEffort=True
).getInfo()

# 8. Extract visualization limits with defaults
air_temp_min = stats.get('NOAA_air_temperature_median_min', -50)
air_temp_max = stats.get('NOAA_air_temperature_median_max', 50)
specific_hum_min = stats.get('NOAA_specific_humidity_median_min', 0)
specific_hum_max = stats.get('NOAA_specific_humidity_median_max', 0.02)
wind_speed_min = stats.get('NOAA_wind_speed_median_min', 0)
wind_speed_max = stats.get('NOAA_wind_speed_median_max', 20)

vis_params = {
    'NOAA_air_temperature_median': {
        'min': air_temp_min, 'max': air_temp_max,
        'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
    },
    'NOAA_specific_humidity_median': {
        'min': specific_hum_min, 'max': specific_hum_max,
        'palette': ['black', 'purple', 'blue', 'green', 'yellow', 'red']
    },
    'NOAA_wind_speed_median': {
        'min': wind_speed_min, 'max': wind_speed_max,
        'palette': ['white', 'blue', 'cyan', 'green', 'yellow', 'red']
    }
}

# 9. Map display
Map_Final = geemap.Map()
Map_Final.centerObject(aoi, 4)

for band in bands:
    band_median = f"NOAA_{band}_median"
    band_max = f"NOAA_{band}_max"
    band_min = f"NOAA_{band}_min"

    Map_Final.addLayer(atmos_median.select(band_median), vis_params[band_median], f"NOAA {band} Median (2012)")
    Map_Final.addLayer(atmos_max.select(band_max), vis_params[band_median], f"NOAA {band} Max (2012)")
    Map_Final.addLayer(atmos_min.select(band_min), vis_params[band_median], f"NOAA {band} Min (2012)")

Map_Final.addLayerControl()
Map_Final


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🌃 2.4 Generate VIIRS Nighttime Lights Composite (2012)

In this sub-step, we generate a composite from the VIIRS Day/Night Band (DNB) monthly radiance product  
(`NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG`) for the year 2012.

This dataset provides global nightlight observations, which can serve as a proxy for urbanization, fishing fleet presence, and economic activity.

The process includes:

1. Filtering VIIRS DNB imagery by AOI and year  
2. Selecting the average radiance band (`avg_rad`)  
3. Computing **median**, **maximum**, and **minimum** composites  
4. Reprojecting and resampling images to a common CRS and scale  
5. Visualizing the result with appropriate color palettes


In [7]:
import ee
import geemap

# 1. Load VIIRS DNB monthly image collection
viirs_collection = (
    ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
)

# 2. Compute pixel-wise statistics (clip to AOI)
viirs_median = viirs_collection.select('avg_rad').median().clip(aoi).resample('bicubic').rename('VIIRS_median')
viirs_max = viirs_collection.select('avg_rad').max().clip(aoi).resample('bicubic').rename('VIIRS_max')
viirs_min = viirs_collection.select('avg_rad').min().clip(aoi).resample('bicubic').rename('VIIRS_min')

# 3. Combine into single composite
viirs_composite = ee.Image.cat([viirs_median, viirs_max, viirs_min])

# 4. Initialize interactive map
Map = geemap.Map()
Map.centerObject(aoi, 4)

# 5. Visualization settings
Map.addLayer(viirs_median, {'min': 0, 'max': 1, 'palette': ['black', 'blue', 'yellow', 'red']}, 'VIIRS Median - 2012')
Map.addLayer(viirs_max, {'min': 0, 'max': 1, 'palette': ['black', 'green', 'yellow', 'red']}, 'VIIRS Max - 2012')
Map.addLayer(viirs_min, {'min': 0, 'max': 1, 'palette': ['black', 'purple', 'cyan', 'white']}, 'VIIRS Min - 2012')

Map.addLayerControl()
Map


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🧭 2.5 Add Bathymetric Data from GEBCO (2012)

In this sub-step, we integrate bathymetric data from the **General Bathymetric Chart of the Oceans (GEBCO)**  
(`projects/sat-io/open-datasets/gebco/gebco_grid`) into our composite for the year 2012.

Bathymetry is a key environmental feature influencing marine ecosystems, ocean productivity, and the spatial behavior of fishing fleets.

The process includes:

1. Loading the GEBCO global bathymetric grid  
2. Clipping to the area of interest (AOI)  
3. Reprojecting the dataset to match the CRS and resolution of our other layers  
4. Adding it to the final composite  
5. Visualizing bathymetry and overlapped radiance data from VIIRS


In [13]:
import ee
import geemap

# Load GEBCO bathymetric grid and clip to AOI
gebco_grid = ee.ImageCollection("projects/sat-io/open-datasets/gebco/gebco_grid") \
    .median() \
    .clip(aoi)

# Resample and rename for consistency (no reprojection needed)
bathymetry = gebco_grid.resample('bicubic') \
    .rename('Bathymetry')

# Create map for visualization
Map_Bathymetry = geemap.Map()
Map_Bathymetry.centerObject(aoi, 4)

# Add GEBCO bathymetry layer to the map
Map_Bathymetry.addLayer(bathymetry, {
    'palette': ['000000', '000080', '00008b', '0000cd', '4682b4', '87ceeb', 'b0e0e6', 'e0ffff'],
    'min': -5000,
    'max': 0
}, 'GEBCO Bathymetry')

# Show map
Map_Bathymetry.addLayerControl()
Map_Bathymetry



Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🌊 2.6 Generate Ocean Current Velocity Composite (2012)

In this sub-step, we use the HYCOM Sea Water Velocity dataset (`HYCOM/sea_water_velocity`) to generate a composite of ocean current magnitude for 2012.

The dataset provides the U (east-west) and V (north-south) components of ocean velocity, which we use to compute the current magnitude using the formula:

**current = √(U² + V²)**

The process includes:

1. Filtering HYCOM data by year and region  
2. Calculating the vector magnitude of ocean currents  
3. Computing **median**, **maximum**, and **minimum** composites  
4. Reprojecting to a standard CRS and resolution  
5. Visualizing current intensity using a diverging color palette



In [9]:
import ee
import geemap

# 1. Load HYCOM ocean current dataset and filter by date
hycom_dataset = ee.ImageCollection("HYCOM/sea_water_velocity").filterDate(start_date, end_date)

# 2. Compute velocity magnitude: √(U² + V²) for each image
ocean_current_magnitude = hycom_dataset.map(lambda img: img.expression(
    "sqrt(pow(u, 2) + pow(v, 2))", {
        'u': img.select('velocity_u_0'),
        'v': img.select('velocity_v_0')
    }).rename('Ocean_Current_Magnitude')
)

# 3. Compute statistics and clip to AOI
ocean_current_median = ocean_current_magnitude.median().clip(aoi).resample('bicubic').rename('Ocean_Current_median')
ocean_current_max = ocean_current_magnitude.max().clip(aoi).resample('bicubic').rename('Ocean_Current_max')
ocean_current_min = ocean_current_magnitude.min().clip(aoi).resample('bicubic').rename('Ocean_Current_min')

# 4. Combine into composite
ocean_current_composite = ee.Image.cat([
    ocean_current_median,
    ocean_current_max,
    ocean_current_min
])

# 5. Visualization settings
currents_vis = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'cyan', 'blue', 'purple']
}

# 6. Initialize map and display
Map_HYCOM = geemap.Map()
Map_HYCOM.centerObject(aoi, 4)

Map_HYCOM.addLayer(ocean_current_median, currents_vis, 'Ocean Current Median (2012)')
Map_HYCOM.addLayer(ocean_current_max, currents_vis, 'Ocean Current Max (2012)')
Map_HYCOM.addLayer(ocean_current_min, currents_vis, 'Ocean Current Min (2012)')

Map_HYCOM.addLayerControl()
Map_HYCOM


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### 🌡️ 2.7 Sea-Level Pressure from NCEP/NCAR Reanalysis (2012)

In this sub-step, we generate a composite of sea-level pressure (SLP) using the  
**NCEP/NCAR Reanalysis dataset** (`NCEP_RE/sea_level_pressure`) for the year 2012.

Sea-level pressure is an important atmospheric driver for wind systems, storms, and ocean-atmosphere interactions,  
which in turn affect marine ecosystems and fishing conditions.

The process includes:

1. Filtering the dataset by date and region  
2. Selecting the `slp` band (sea-level pressure)  
3. Computing **median**, **maximum**, and **minimum** values  
4. Reprojecting data to a common CRS and resolution  
5. Visualizing the results with dynamic color scaling


In [10]:
import ee
import geemap

# 1. Load NCEP/NCAR Reanalysis SLP dataset and filter by date
slp_dataset = ee.ImageCollection("NCEP_RE/sea_level_pressure")
slp_2012 = slp_dataset.filterDate(start_date, end_date).select('slp')

# 2. Compute statistics and clip to AOI
slp_median = slp_2012.median().clip(aoi).resample('bicubic').rename("NCEP_SLP_median")
slp_max = slp_2012.max().clip(aoi).resample('bicubic').rename("NCEP_SLP_max")
slp_min = slp_2012.min().clip(aoi).resample('bicubic').rename("NCEP_SLP_min")

# 3. Combine into composite
slp_composite = ee.Image.cat([slp_median, slp_max, slp_min])

# 4. Compute visualization stats
stats = slp_median.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=scale_value,
    bestEffort=True
).getInfo()

# 5. Extract values with default fallback
slp_min_val = stats.get('NCEP_SLP_median_min', 98000)
slp_max_val = stats.get('NCEP_SLP_median_max', 103000)

# 6. Define visualization parameters
slp_vis = {
    'min': slp_min_val,
    'max': slp_max_val,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']
}

# 7. Initialize and display map
Map_SLP = geemap.Map()
Map_SLP.centerObject(aoi, 4)

Map_SLP.addLayer(slp_median, slp_vis, "NCEP Sea Level Pressure Median (2012)")
Map_SLP.addLayer(slp_max, slp_vis, "NCEP Sea Level Pressure Max (2012)")
Map_SLP.addLayer(slp_min, slp_vis, "NCEP Sea Level Pressure Min (2012)")

Map_SLP.addLayerControl()
Map_SLP


Map(center=[59.50338445527181, 180], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

## 🧩 Assemble Final Composite

Now that we have generated and standardized all environmental and satellite-based layers,  
we can combine them into a single multiband image called `final_composite`.

This composite includes:

- MODIS Surface Reflectance  
- MODIS-Aqua Ocean Color  
- NOAA Atmospheric Properties  
- VIIRS Nighttime Lights  
- GEBCO Bathymetry  
- HYCOM Ocean Current Magnitude  
- NCEP/NCAR Sea-Level Pressure



In [14]:
# ===============================================
# 🧩 Assemble Final Composite
# ===============================================
# Make sure all *_composite and bathymetry objects are already defined in memory

final_composite = ee.Image.cat([
    sr_composite,               # MODIS Surface Reflectance
    aqua_composite,             # MODIS-Aqua Ocean Color
    atmos_composite,            # NOAA Near-Surface Atmosphere
    viirs_composite,            # VIIRS Nighttime Lights
    bathymetry,                 # GEBCO Bathymetry
    ocean_current_composite,    # HYCOM Ocean Current Magnitude
    slp_composite               # NCEP/NCAR Sea-Level Pressure
])

print("✅ Final composite assembled successfully.")


# 1. Retrieve metadata from final_composite
band_names = final_composite.bandNames().getInfo()
num_bands = len(band_names)


# 2. Print metadata and sampled results
print(f"Number of bands in final_composite: {num_bands}")
print(f"Band names: {band_names}")

✅ Final composite assembled successfully.
Number of bands in final_composite: 88
Band names: ['sur_refl_b08_median', 'sur_refl_b09_median', 'sur_refl_b10_median', 'sur_refl_b11_median', 'sur_refl_b12_median', 'sur_refl_b13_median', 'sur_refl_b14_median', 'sur_refl_b15_median', 'sur_refl_b16_median', 'sur_refl_b08_max', 'sur_refl_b09_max', 'sur_refl_b10_max', 'sur_refl_b11_max', 'sur_refl_b12_max', 'sur_refl_b13_max', 'sur_refl_b14_max', 'sur_refl_b15_max', 'sur_refl_b16_max', 'sur_refl_b08_min', 'sur_refl_b09_min', 'sur_refl_b10_min', 'sur_refl_b11_min', 'sur_refl_b12_min', 'sur_refl_b13_min', 'sur_refl_b14_min', 'sur_refl_b15_min', 'sur_refl_b16_min', 'MODIS_Aqua_chlor_a_median', 'MODIS_Aqua_nflh_median', 'MODIS_Aqua_poc_median', 'MODIS_Aqua_sst_median', 'MODIS_Aqua_Rrs_412_median', 'MODIS_Aqua_Rrs_443_median', 'MODIS_Aqua_Rrs_469_median', 'MODIS_Aqua_Rrs_488_median', 'MODIS_Aqua_Rrs_531_median', 'MODIS_Aqua_Rrs_547_median', 'MODIS_Aqua_Rrs_555_median', 'MODIS_Aqua_Rrs_645_median', 'M

In [15]:
import ee

# Create export task to save the image as a GEE asset
export_task = ee.batch.Export.image.toAsset(
    image=final_composite,
    description='Export_FinalComposite_Pollock_2012',
    assetId='projects/ee-batistahpedro/assets/Pollock/final_composite_2012',
    region=aoi,
    scale=scale_value,
    crs=crs_code,
    maxPixels=1e13
)

# Start the export
export_task.start()


The `final_composite` image has been successfully assembled and exported as a permanent Earth Engine Asset:

``projects/ee-batistahpedro/assets/Pollock/final_composite_2012``

---

**Note:** This composite was exported to a private Earth Engine asset. If you are reproducing this workflow, you must:
1. Create your own asset folder under your GEE account
2. Update the asset path in the export or import code accordingly

Alternatively, request access to this asset if it's intended to be shared.

---

## Next Steps

The next stages of the workflow — including **feature extraction**, **training data preparation**, **model training**, and **mapping predictions** — will be conducted in a separate notebook:

**Notebook Title:** `Pollock_Modeling.ipynb`

This ensures modular organization and clear separation between data preprocessing (this notebook) and model development (next notebook).
